# S2W5D3: 多头机制 - 专家团队的诞生

如果说昨天的内容是“一个人在读书”，今天的代码就是 **“一个专家团队在开会”**。这是 Transformer 强大的根本原因。

## 1 核心逻辑：从单头到多头

### 1.1 为什么需要多头？

在面试中可能会被问到：“为什么不把维度做大一点，非要搞成多头？”
- **单头 (Single Head)**: 就像你看一张照片，只能聚焦在一个点上。
- **多头 (Multi-Head)**: 就像你戴了 AR 眼镜，同时开启了 8 个窗口：
    - **Head 1**: 关注**语法结构**（主语找谓语）。
    - **Head 2**: 关注**指代关系**（“它”是指前面的哪个人）。
    - **Head 3**: 关注**时序信息**（过去还是未来）。
    - ....
    -  **结论**: 它们将输入特征切分成多个子空间 (Subspaces)，让模型并行地捕捉不同维度的特征。

### 1.2 数据流图 (The Pipeline)

1.  **输入**: $X$ `[Batch, Seq, D_model]`
2.  **分发 (Linear Projections)**: 通过 $W_q, W_k, W_v$ 变换，并切分成 `n_heads` 个部分。
3.  **并行计算**: 8 个头同时做 Self-Attention（复用昨天的函数）。
4.  **拼接 (Concat)**: 把 8 个头的结果拼回去。
5.  **融合 (Final Linear)**: 通过 $W_o$ 把拼接后的结果混合一次。

## 2 代码实现

这一段代码是 Transformer 模型的 **骨架**。请仔细敲入，重点关注 `view` (重塑形状) 和 `permute` (交换维度) 的操作。

In [1]:
import os
import sys
import torch
import torch.nn as nn

# --- 设置路径，确保能扫描到 src 目录 ---
# 获取当前 notebook 所在的文件夹 (week5)
current_dir = os.getcwd()
# 获取项目根目录 (假设 structure 是 project/stage2/week5，需要往上跳两级)
# 这一步根据你的实际路径可能需要微调，通常把 project_root 加入 path 最稳妥
project_root = os.path.abspath(os.path.join(current_dir, "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.transformer_components import scaled_dot_product_attention

class MultiHeadAttention(nn.Module):
    """
    Day 3: 多头注意力 (Multi-Head Attention)
    """
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"
        
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)
        
    def split_heads(self, x, batch_size):
        """
        [Batch, Seq, D_model] -> [Batch, Heads, Seq, D_k]
        """
        x = x.view(batch_size, -1, self.n_heads, self.d_k)
        return x.permute(0, 2, 1, 3)

    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)
        
        # 1. Linear Projections + Split Heads
        Q = self.split_heads(self.w_q(q), batch_size)
        K = self.split_heads(self.w_k(k), batch_size)
        V = self.split_heads(self.w_v(v), batch_size)
        
        # 2. Scaled Dot-Product Attention (复用上面的函数)
        attention_output, attention_weights = scaled_dot_product_attention(Q, K, V, mask)
        
        # 3. Concat Heads
        # [Batch, Heads, Seq, D_k] -> [Batch, Seq, Heads, D_k]
        attention_output = attention_output.permute(0, 2, 1, 3)
        # Flatten: [Batch, Seq, D_model]
        attention_output = attention_output.contiguous().view(batch_size, -1, self.d_model)
        
        # 4. Final Linear
        output = self.w_o(attention_output)
        
        return output, attention_weights

## 3 代码细节解析 (Code Walkthrough)

这里有几个 PyTorch 的**暗坑**，必须搞清楚：

### 3.1 `view` vs `reshape`

  * 代码里用了 `view`。`view` 只能处理**内存连续 (Contiguous)** 的张量。
  * 如果我们做过 `permute` 或 `transpose`，张量在内存里就乱了（不再连续）。
  * 所以在 Step 3.2，我们必须先调用 **`.contiguous()`** 把内存理顺，然后再 `view`，否则必报错。

#### 3.2  为什么要最后加一个 `w_o` (Output Linear)？

  * **物理直觉**: 8 个专家开完会，每个人都给出了自己的结论（Concat 后的结果）。
  * **W\_o 的作用**: 这是一个 **“会议记录员”** 或 **“决策者”**。它负责把这 8 个专家的意见**加权融合**，生成最终的总结报告。
  * **具身视角**: 假如 Head 1 看到了红色，Head 2 看到了圆形。拼接后是 `[红, 圆]`。`W_o` 负责把它们融合成概念 `[苹果]`。

## 4 验证环节 (Verify)

In [2]:
# --- 测试代码 ---
d_model = 512
n_heads = 8
seq_len = 10
batch_size = 4

# 1. 实例化
mha = MultiHeadAttention(d_model, n_heads)

# 2. 创建假数据
# 在 Self-Attention 中，Q=K=V=X
x = torch.randn(batch_size, seq_len, d_model)

# 3. 前向传播
output, weights = mha(x, x, x)

print("Input shape: ", x.shape)
print("Output shape:", output.shape) # 必须也是 [4, 10, 512]
print("Weights shape:", weights.shape) # [4, 8, 10, 10] (Batch, Heads, Seq, Seq)

# 验证输入输出维度是否一致
assert output.shape == x.shape
print("✅ 测试通过！维度变换完美。")

Input shape:  torch.Size([4, 10, 512])
Output shape: torch.Size([4, 10, 512])
Weights shape: torch.Size([4, 8, 10, 10])
✅ 测试通过！维度变换完美。


## 5\. 面试必问 & 具身展望

### 🎤 面试高频题

**Q: Multi-Head Attention 的计算复杂度是多少？和 Single-Head 一样吗？**

  * **A**: **几乎一样。**
      * 虽然变成了 8 个头，但每个头的维度降到了 $1/8$ ($64$ vs $512$)。
      * 总的计算量（矩阵乘法）主要取决于总维度 $d_{model}$。
      * $W_q, W_k, W_v$ 的参数量和把它们切开算是一样的。
      * *微小的区别在于并行计算的开销，但在理论 FLOPS 上是一致的。*

### 🤖 具身智能 (Embodied AI) 连接

想象你正在做一个 **多模态机器人 (Robot Transformer)**：

  * **Head 1-4**: 负责处理 **视觉 (Vision)** —— 看着前面的路。
  * **Head 5-8**: 负责处理 **本体感知 (Proprioception)** —— 感受手臂的关节角度。
  * **Concat**: 在 Transformer 内部，视觉信息和关节信息被“拼接”在一起。
  * **Attention**: 机器人学会了：“当我看到那个杯子 (Vision) 时，我的手臂应该处于抓取状态 (Proprioception)。”

这就是为什么 Transformer 能通吃 CV 和 NLP，也能控制机器人的原因——**Multi-Head 机制天生适合融合多源信息。**

### 🚀 今日任务

1.  **代码实现**: 亲手写一遍 `MultiHeadAttention` 类，特别是 `split_heads` 里的维度变换。
2.  **Debug**: 试着去掉 `.contiguous()`，看看报错长什么样（增加 Debug 经验）。
3.  **理解**: 盯着测试代码的输出形状，确认它和输入一模一样。这就是 Transformer Block 的\*\*“积木特性”\*\*——输入输出同型，所以可以无限堆叠！

写完后，请告诉我“**MHA 搞定**”，明天我们要把这些积木搭成高楼：**Encoder Layer 和 Residual Connection**。